<a href="https://colab.research.google.com/github/thikhamporn0589-bit/Colab/blob/main/Lab%20%20Pan_sharpening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pan-sharpening**

เทคนิค Pan-sharpening เป็นเทคนิคที่ใช้กับภาพถ่ายดาวเทียม ที่มี Panchromatic band เช่น THEOS-1, LANDSAT-8, LANDSAT-9 เพื่อให้มีรายละเอียดและมีประโยชน์มากขึ้น โดย Lab นี้เราจะทดลองกับข้อมูลดาวเทียม Landsat-9

ข้อมูลถ่ายที่เป็น Multi-Spectral มักจะมีรายละเอียดภาพไม่ค่อยดีนัก เมื่อเทียบกับข้อมูล Panchromatics  ดังนั้น การปรับความคมชัดของภาพช่วยได้โดยการรวมภาพเหล่านี้เข้ากับภาพขาวดำพิเศษที่มีรายละเอียดมากขึ้น ภาพขาวดำที่เรียกว่าภาพแพนโครมาติก มีรายละเอียดในระดับที่สูงกว่า แต่ไม่มีสี เราต้องการใช้รายละเอียดจากภาพแพนโครมาติกและสีจากภาพ Landsat-9 เพื่อสร้างภาพสีที่มีรายละเอียดสูงใหม่

ในการดำเนินการใน Lab นี้ นักศึกษา จำเป็นต้องตรวจสอบให้แน่ใจว่าข้อมูลทั้ง Panchromatic และ Multispectral ของ Landsat-9 อยู่ในตำแหน่งที่สมบูรณ์แบบ เพื่อให้เข้ากันได้อย่างลงตัว

มีหลายวิธีในการรวมภาพเหล่านี้ เช่น Brovey Transform, IHS Transform และ PCA วิธีการเหล่านี้ทำให้แน่ใจว่าภาพใหม่จะคงสีจาก Landsat-9 แต่ยังได้รับรายละเอียดเพิ่มเติมจากภาพแพนโครมาติกด้วย


เมื่อเราทำ Pan-sharpening เสร็จแล้ว เราก็จะได้ภาพใหม่ที่เป็นภาพสีที่มีรายละเอียดสูงขึ้น  โดยมีสีทั้งหมดจาก Landsat-9 และความคมชัดพิเศษจากภาพแบบแพนโครมาติก รูปภาพใหม่นี้สามารถนำไปใช้ได้หลายอย่าง เช่น ศึกษาพื้นดิน ค้นหาการเปลี่ยนแปลง หรือเพียงแค่ดูรายละเอียดเพิ่มเติมเกี่ยวกับโลก ดังนั้น การปรับความคมชัดของภาพเป็นวิธีหนึ่งในการทำให้ภาพจากดาวเทียมมีประโยชน์มากขึ้นสำหรับงานสำคัญทุกประเภท

In [1]:
# ทำการ Authenticate และ initialize Earth Engine
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-thikhamporn0589') #อย่าลืมเปลี่ยนชื่อโปรเจคของตัวเอง

In [2]:
# กำหนดพื้นที่สนใจ (เชียงใหม่)
geometry = ee.Geometry.Point([98.95799098999555, 18.84423947416328])

# เรียกภาพ Landsat 9 ที่เมฆน้อยกว่า 10%
image = (ee.ImageCollection("LANDSAT/LC09/C02/T1_TOA")
         .filterDate('2022-01-01', '2022-03-30')
         .filterBounds(geometry)
         .filter(ee.Filter.lt('CLOUD_COVER', 10))   # กรองเมฆ
         .sort('CLOUD_COVER')                       # เรียงจากเมฆน้อยสุด
         .first())

print('Image ID:', image.get('LANDSAT_PRODUCT_ID').getInfo())


Image ID: LC09_L1TP_131047_20220214_20230427_02_T1


In [3]:
# ----------------------------
# 1) เลือก RGB และ Pan band
# ----------------------------
rgb = image.select(['B4', 'B3', 'B2'])   # 30m
pan = image.select('B8')                 # 15m


# ----------------------------
# 2) ปรับความละเอียด RGB ให้เท่ากับ Pan (15m)
# ----------------------------
rgb_resampled = rgb.resample('bicubic').reproject(
    crs=pan.projection(),
    scale=15
)


# ----------------------------
# 3) แปลง RGB เป็น HSV
# ----------------------------
hsv = rgb_resampled.rgbToHsv()


# ----------------------------
# 4) แทนที่ Value ด้วย Pan band
# ----------------------------
huesat = hsv.select(['hue', 'saturation'])
upres = ee.Image.cat([huesat, pan]).hsvToRgb()


# ----------------------------
# 5) ตั้งชื่อ band ใหม่ (กัน error ตอน export)
# ----------------------------
upres = upres.rename(['B4', 'B3', 'B2'])


In [4]:
# ----------------------------
# 1) กำหนดพื้นที่แสดงผล
# ----------------------------
region = geometry.buffer(1000).bounds()

rgb_clip = rgb.clip(region)
pan_clip = pan.clip(region)
upres_clip = upres.clip(region)


# ----------------------------
# 2) สร้างแผนที่
# ----------------------------
import geemap

Map = geemap.Map(
    center=[18.84423947416328, 98.95799098999555],
    zoom=14
)

# กำหนดค่าการแสดงผลให้เหมาะกับ TOA
vis_rgb = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.3
}

vis_pan = {
    'min': 0,
    'max': 0.3
}

# ----------------------------
# 3) เพิ่ม Layer เปรียบเทียบ
# ----------------------------
Map.addLayer(rgb_clip, vis_rgb, 'Original RGB (30m)')
Map.addLayer(pan_clip, vis_pan, 'Panchromatic (15m)')
Map.addLayer(upres_clip, vis_rgb, 'Pan-sharpened (15m)')

Map


Map(center=[18.84423947416328, 98.95799098999555], controls=(WidgetControl(options=['position', 'transparent_b…

คำถาม
ข้อเพื่อทดสอบความเข้าใจของนักศึกษาเกี่ยวกับเทคนิคการทำ Pan-sharpening ด้วยภาพ Landsat-9 จงตอบคำถามต่อไปนี้

1. อะไรคือเป้าหมายหลักของการปรับความคมชัดของภาพในการสำรวจระยะไกล โดยเฉพาะเมื่อใช้ภาพ Landsat-9 อธิบายว่าเหตุใดจึงมีความสำคัญในการประมวลผลภาพ


เป้าหมายหลักของการทำ Pan-sharpening คือการทำให้ภาพสีจาก Landsat-9 มีความคมชัดมากขึ้น โดยเอาความละเอียดสูงจากภาพ Panchromatic (15 เมตร) มารวมกับภาพสี Multispectral (30 เมตร)

เพราะภาพ Multispectral มีสีครบแต่รายละเอียดไม่คม ส่วน Panchromatic คมแต่ไม่มีสี เราเลยรวมสองอย่างเข้าด้วยกัน เพื่อให้ได้ภาพสีที่ทั้งคมและดูรายละเอียดได้ชัดขึ้น ซึ่งสำคัญมากในการวิเคราะห์พื้นที่ เช่น ดูสิ่งปลูกสร้าง ถนน หรือการใช้ที่ดิน


2.อธิบายขั้นตอนสำคัญที่เกี่ยวข้องกับการปรับความคมชัดด้วนเทคนิค Pan-sharpening ตั้งแต่การรับข้อมูลไปจนถึงการสร้างภาพที่ปรับความคมชัด เทคนิค Pan-sharpening มี กระบวนการสุ่มตัวอย่างใหม่ช่วยจัดแนวภาพหลายสเปกตรัมและภาพแพนโครมาติกอย่างไร

ขั้นตอนหลัก ๆ คือ

เลือกภาพ Landsat-9 และแยก RGB (B4, B3, B2) กับ Pan (B8)

ปรับความละเอียดของ RGB จาก 30m ให้เป็น 15m ด้วยการ resample เพื่อให้ตรงกับ Pan

แปลงภาพ RGB เป็นระบบสี HSV

เอา Hue กับ Saturation ไว้ แล้วแทนที่ค่า Value ด้วยภาพ Pan

แปลงกลับเป็น RGB จะได้ภาพสีที่คมชัดขึ้น

Resampling มีหน้าที่ทำให้ภาพ Multispectral กับ Panchromatic มีขนาดพิกเซลและตำแหน่งตรงกัน ถ้าไม่ทำขั้นตอนนี้ ภาพจะไม่ตรงกันและรวมกันไม่ได้อย่างถูกต้อง